In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import math
from tqdm import tqdm
from functions import *
from tickers_list import *
sns.set_style('whitegrid')

In [6]:
stocks = ["LUPIN.NS", "VOLTAS.NS", "YESBANK.NS"]

INITIAL_AMT_INVESTED = 10000
NUM_OF_SIMULATIONS = 5000
start_date='2023-01-01'
end_date='2023-12-31'

In [8]:
## GATHER STOCK PRICES

stock_prices, stock_dividend = fetch_data(stocks, start_date='2023-01-01', end_date='2023-12-31')

## CALCULATE LOG RETURNS

log_returns = np.log(stock_prices/stock_prices.shift(1))[1:]

## DOWNSIDE DEVIATION (SORTINO RATIO)

log_returns_down_only = log_returns.applymap(lambda x: 0 if x>0 else x)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:30<00:00, 10.30s/it]


In [11]:
portfolio_weights = []
portfolio_num_of_shares = []
portfolio_returns = []
portfolio_risks = []
portfolio_risks_sortino = []
portfolio_value = []
portfolio_beta = []
NUM_TRADING_DAYS = len(stock_prices)

nifty50_price,temp = fetch_data("^NSEI", start_date=start_date, end_date=end_date)
log_returns_market = np.log(nifty50_price/nifty50_price.shift(1))[1:]

## MONTE CARLO SIMULATION

for i in tqdm(range(NUM_OF_SIMULATIONS)):
    ## STEP-I: generating random weights
    w = np.random.rand(log_returns.shape[1])
    w = np.expand_dims(w, axis=0)
    w = w/np.sum(w)
    portfolio_weights.append(w.flatten())

    ## STEP-II: calculating returns using average annual returns
    p_returns = (log_returns.mean() @ w.T*NUM_TRADING_DAYS)[0]
    portfolio_returns.append(p_returns)

    ## STEP-III: calculating risks
    p_risks = np.sqrt(w @ log_returns.cov()*NUM_TRADING_DAYS @ w.T)[0][0]
    portfolio_risks.append(p_risks)
    
    p_risks = np.sqrt(w @ log_returns_down_only.cov()*NUM_TRADING_DAYS @ w.T)[0][0]
    portfolio_risks_sortino.append(p_risks)

    ## CALCULATING PORTFOLIO VALUE
    num_of_shares = INITIAL_AMT_INVESTED*w[0]/stock_prices.iloc[0,:]
    num_of_shares = np.expand_dims(num_of_shares, axis=0)
    portfolio_num_of_shares.append(num_of_shares)

    portfolio_value_i = np.sum(num_of_shares*stock_prices, axis=1)
    portfolio_value.append(portfolio_value_i)

    log_returns_portfolio_value_i = np.log(portfolio_value_i/portfolio_value_i.shift(1))[1:]

    ## STEP-IV: calculating portfolio beta
    beta = calculate_beta(log_returns_portfolio_value_i, log_returns_market)
    portfolio_beta.append(beta)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:12<00:00, 406.52it/s]


In [45]:
portfolio_weights = np.array(portfolio_weights)
portfolio_num_of_shares = np.array(portfolio_num_of_shares)
portfolio_returns = np.array(portfolio_returns)
portfolio_risks = np.array(portfolio_risks)
portfolio_risks_sortino = np.array(portfolio_risks_sortino)
portfolio_value = np.array(portfolio_value)
portfolio_beta = np.array(portfolio_beta)



# taking IR of government issued 10-year bonds as risk-free rate
risk_free_rate = 0.07

## CALCULATING RATIOS

sharpe_ratios = (portfolio_returns - risk_free_rate) / portfolio_risks
sortino_ratios = (portfolio_returns - risk_free_rate) / portfolio_risks_sortino

## TREYNOR RATIO
treynor_ratios = (portfolio_returns - risk_free_rate) / portfolio_beta

## JENSONS ALPHA
CAPM_expected_return = risk_free_rate + beta*(np.sum(log_returns_market) - risk_free_rate)
CAPM_expected_return = CAPM_expected_return[0]
jenson_alpha = portfolio_returns - CAPM_expected_return

In [46]:
## CREATE A TABLE FOR ALL THE SIMULATIONS

df_sims = pd.DataFrame({'Portfolio_return':portfolio_returns,'Portfolio_risk':portfolio_risks,
                        'Sharpe':sharpe_ratios,'Sortino':sortino_ratios,'Treynor':treynor_ratios,'JensonAlpha':jenson_alpha})
df_sims = pd.concat([df_sims,weights_to_dataframe(portfolio_weights, stocks)], axis=1)

In [48]:
df_sims

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
0,0.353330,0.167105,1.695522,2.858490,0.577484,0.217580,0.393935,0.579358,0.026707
1,0.175213,0.180485,0.582949,0.966186,0.175189,0.039464,0.162785,0.401901,0.435314
2,0.269342,0.168051,1.186203,1.994422,0.333424,0.133593,0.349534,0.312023,0.338443
3,0.224421,0.188787,0.817964,1.376352,0.240247,0.088671,0.307145,0.222038,0.470816
4,0.077297,0.242292,0.030118,0.050315,0.010313,-0.058452,0.066433,0.217007,0.716559
...,...,...,...,...,...,...,...,...,...
4995,0.345232,0.161851,1.700528,2.886689,0.487034,0.209482,0.462604,0.346162,0.191233
4996,0.247895,0.166988,1.065316,1.781039,0.301875,0.112145,0.297663,0.359001,0.343336
4997,0.250434,0.190577,0.946778,1.602450,0.277320,0.114685,0.367544,0.172209,0.460246
4998,0.118138,0.188835,0.254922,0.418935,0.085813,-0.017611,0.002349,0.590605,0.407046


In [49]:
sharpe_max_indices = np.argsort(sharpe_ratios)[-1:-11:-1]
sortino_max_indices = np.argsort(sortino_ratios)[-1:-11:-1]
treynor_max_indices = np.argsort(treynor_ratios)[-1:-11:-1]
jenson_alpha_max_indices = np.argsort(jenson_alpha)[-1:-11:-1]

In [54]:
df_sims.iloc[sharpe_max_indices,:]

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
27,0.535036,0.210904,2.204959,3.842729,0.797977,0.399286,0.843419,0.152847,0.003735
3633,0.551500,0.218444,2.204229,3.847237,0.814525,0.415750,0.883509,0.116011,0.000480
3693,0.544520,0.215606,2.200859,3.839921,0.804726,0.408770,0.868817,0.125076,0.006108
1584,0.511252,0.200623,2.199414,3.822438,0.774253,0.375502,0.784456,0.209041,0.006504
2040,0.520063,0.204655,2.199127,3.827211,0.780191,0.384314,0.808769,0.181202,0.010029
4595,0.502104,0.196754,2.196157,3.811154,0.767154,0.366354,0.759799,0.236277,0.003924
932,0.519152,0.204645,2.194779,3.820347,0.776105,0.383402,0.809129,0.175907,0.014964
837,0.499478,0.195697,2.194601,3.806801,0.764978,0.363728,0.752800,0.243871,0.003329
2689,0.506177,0.198776,2.194307,3.811851,0.766725,0.370427,0.773768,0.215645,0.010587
2361,0.534040,0.211533,2.193703,3.825277,0.789173,0.398290,0.847352,0.136997,0.015651


In [55]:
df_sims.iloc[treynor_max_indices,:]

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
105,0.576575,0.231565,2.187619,3.826645,0.833178,0.440826,0.948847,0.047743,0.003410
438,0.575474,0.234333,2.157075,3.777092,0.815414,0.439724,0.959688,0.011758,0.028554
1622,0.561822,0.224941,2.186449,3.821978,0.815110,0.426072,0.916251,0.071289,0.012461
3633,0.551500,0.218444,2.204229,3.847237,0.814525,0.415750,0.883509,0.116011,0.000480
599,0.565017,0.227777,2.173256,3.801535,0.811494,0.429267,0.929444,0.048749,0.021807
468,0.555423,0.221798,2.188586,3.823876,0.809037,0.419674,0.900588,0.085836,0.013576
1592,0.563309,0.227702,2.166462,3.790244,0.805958,0.427559,0.928501,0.043426,0.028074
3693,0.544520,0.215606,2.200859,3.839921,0.804726,0.408770,0.868817,0.125076,0.006108
902,0.552211,0.220862,2.183311,3.814728,0.802365,0.416461,0.895658,0.084801,0.019541
27,0.535036,0.210904,2.204959,3.842729,0.797977,0.399286,0.843419,0.152847,0.003735


In [57]:
df_sims.iloc[sortino_max_indices,:]

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
3633,0.551500,0.218444,2.204229,3.847237,0.814525,0.415750,0.883509,0.116011,0.000480
27,0.535036,0.210904,2.204959,3.842729,0.797977,0.399286,0.843419,0.152847,0.003735
3693,0.544520,0.215606,2.200859,3.839921,0.804726,0.408770,0.868817,0.125076,0.006108
2040,0.520063,0.204655,2.199127,3.827211,0.780191,0.384314,0.808769,0.181202,0.010029
105,0.576575,0.231565,2.187619,3.826645,0.833178,0.440826,0.948847,0.047743,0.003410
2361,0.534040,0.211533,2.193703,3.825277,0.789173,0.398290,0.847352,0.136997,0.015651
468,0.555423,0.221798,2.188586,3.823876,0.809037,0.419674,0.900588,0.085836,0.013576
1584,0.511252,0.200623,2.199414,3.822438,0.774253,0.375502,0.784456,0.209041,0.006504
1622,0.561822,0.224941,2.186449,3.821978,0.815110,0.426072,0.916251,0.071289,0.012461
932,0.519152,0.204645,2.194779,3.820347,0.776105,0.383402,0.809129,0.175907,0.014964


In [58]:
df_sims.iloc[jenson_alpha_max_indices,:]

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
105,0.576575,0.231565,2.187619,3.826645,0.833178,0.440826,0.948847,0.047743,0.003410
438,0.575474,0.234333,2.157075,3.777092,0.815414,0.439724,0.959688,0.011758,0.028554
599,0.565017,0.227777,2.173256,3.801535,0.811494,0.429267,0.929444,0.048749,0.021807
1592,0.563309,0.227702,2.166462,3.790244,0.805958,0.427559,0.928501,0.043426,0.028074
1622,0.561822,0.224941,2.186449,3.821978,0.815110,0.426072,0.916251,0.071289,0.012461
468,0.555423,0.221798,2.188586,3.823876,0.809037,0.419674,0.900588,0.085836,0.013576
902,0.552211,0.220862,2.183311,3.814728,0.802365,0.416461,0.895658,0.084801,0.019541
3633,0.551500,0.218444,2.204229,3.847237,0.814525,0.415750,0.883509,0.116011,0.000480
2753,0.550114,0.223927,2.144061,3.751070,0.780119,0.414365,0.907452,0.041440,0.051107
1205,0.548268,0.220361,2.170390,3.793144,0.790911,0.412519,0.892341,0.075757,0.031902
